In [1]:
import os
import pymysql
from dotenv import load_dotenv
load_dotenv()

HOST = os.getenv('HOST')
USER = os.getenv('MYSQL_USER')
PASSWORD = os.getenv('MYSQL_ROOT_PASSWORD')
DB = os.getenv('MYSQL_DATABASE')

connection = pymysql.connect(host=HOST, 
                      user = USER,
                     password = PASSWORD,
                     db = DB)

cursor = connection.cursor()

## Attribute Selection
* __Dropping unecessary columns__

In [51]:
cursor.execute("SELECT * FROM pedidos")
cursor.fetchall()[0]

"""
Caption:
numero
data_pedido
hora_pedido
cliente
endereco
telefone
tipo_entrega
valor_pizza
valor_borda
valor_refrigerante
valor_entrga
valor_total
hora_entrega
tempo
"""

(1,
 '2011-10-07',
 '18:45:00',
 'Cliente 1',
 'Endereco 1',
 '00-000-0000',
 'Buscar',
 32.0,
 0.0,
 0.0,
 0.0,
 32.0,
 '19:15:00',
 '00:30:00')

In [2]:
#Defining the columns that are going to be droped
numero = "ALTER TABLE pedidos DROP COLUMN numero;"
cliente = "ALTER TABLE pedidos DROP COLUMN cliente"
endereco = "ALTER TABLE pedidos DROP COLUMN endereco"
telefone = "ALTER TABLE pedidos DROP COLUMN telefone"
valor_pizza = "ALTER TABLE pedidos DROP COLUMN valor_pizza"
valor_entrega = "ALTER TABLE pedidos DROP COLUMN valor_entrega"
hora_entrega = "ALTER TABLE pedidos DROP COLUMN hora_entrega"

In [3]:
lista = [numero, cliente, endereco, telefone,
        valor_pizza, valor_entrega, 
        hora_entrega]

In [4]:
for i in range(len(lista)):
    cursor.execute(lista[i])
connection.commit()

In [7]:
cursor.execute("SELECT dayname(data_pedido) FROM pedidos")
cursor.fetchall()[0:10]

(('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',),
 ('Friday',))

In [12]:
cursor.execute("SELECT max(data_pedido), min(data_pedido) FROM pedidos")
cursor.fetchone()

(datetime.date(2015, 2, 3), datetime.date(2011, 10, 7))

>__As from this database the entities with data_peido >= 2015 are only a test without value, I am going to remove them__

In [13]:
delete_data = "DELETE FROM pedidos WHERE year(data_pedido) >= 2015"
cursor.execute(delete_data)
connection.commit()

In [15]:
cursor.execute("SELECT max(data_pedido), min(data_pedido) FROM pedidos")
cursor.fetchone()

(datetime.date(2012, 2, 24), datetime.date(2011, 10, 7))

In [2]:
cursor.execute("SELECT data_pedido, dayname(data_pedido) FROM pedidos")
cursor.fetchall()[0:5]

((datetime.date(2011, 10, 7), 'Friday'),
 (datetime.date(2011, 10, 7), 'Friday'),
 (datetime.date(2011, 10, 7), 'Friday'),
 (datetime.date(2011, 10, 7), 'Friday'),
 (datetime.date(2011, 10, 7), 'Friday'))

In [56]:
#Returning time and dayname
cursor.execute("SELECT DATE_FORMAT(hora_pedido, '%H:%i'), dayname(hora_pedido) FROM pedidos")
cursor.fetchall()[0:5]

(('18:45', 'Tuesday'),
 ('18:47', 'Tuesday'),
 ('18:49', 'Tuesday'),
 ('18:50', 'Tuesday'),
 ('18:52', 'Tuesday'))